In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import os

In [2]:
dataset_files = "/content/drive/MyDrive/data_pa/dataset-myimmo.csv"
df_immo = pd.read_csv(dataset_files)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Appartement
df_immo_appart = df_immo[ (df_immo["nature_mutation"]=="Vente") & (df_immo["type_local"]=="Appartement") ]
# Maison
df_immo_maison = df_immo[ (df_immo["nature_mutation"]=="Vente") & (df_immo["type_local"]=="Maison") ]
df_immo_maison

,id_mutation,date_mutation,nature_mutation,code_postal,code_commune,code_region,code_departement,type_local,latitude,longitude,valeur_fonciere,surface_reelle_bati,nombre_pieces_principales,prix_metre_carre
0,2015-1131554,2015-06-25,Vente,97136.0,97130,1,971,Maison,15.848188,-61.644636,1860.00,90.0,4.0,21
2,2015-1131605,2015-07-08,Vente,97136.0,97130,1,971,Maison,15.848572,-61.644027,1570.00,98.0,4.0,16
3,2015-1132114,2015-12-23,Vente,97136.0,97130,1,971,Maison,15.848833,-61.643867,1450.00,148.0,5.0,10
4,2015-1131405,2015-04-09,Vente,97137.0,97131,1,971,Maison,15.862501,-61.597274,720000.00,88.0,3.0,8182
5,2015-1132113,2015-12-30,Vente,97137.0,97131,1,971,Maison,15.862801,-61.586874,10671.43,38.0,3.0,281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071014,2020-227595,2020-12-31,Vente,20600.0,2B120,94,2B,Maison,42.649121,9.443310,375000.00,138.0,5.0,2717
3071015,2020-227568,2020-11-16,Vente,20200.0,2B353,94,2B,Maison,42.714854,9.427883,180000.00,104.0,3.0,1731
3071016,2020-227575,2020-12-17,Vente,20200.0,2B305,94,2B,Maison,42.733560,9.460780,123500.00,33.0,1.0,3742
3071017,2020-227564,2020-10-14,Vente,20233.0,2B224,94,2B,Maison,42.840266,9.456825,265000.00,82.0,4.0,3232


Modele BallTree pour les appartements - 1 modéle par region

In [4]:
from sklearn.neighbors import BallTree
import numpy as np
import time

In [11]:
df_immo_maison['distance_moyenne']=np.zeros(len(df_immo_maison))
df_immo_maison['index_voisins']=np.zeros(len(df_immo_maison))
df_immo_maison

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id_mutation,date_mutation,nature_mutation,code_postal,code_commune,code_region,code_departement,type_local,latitude,longitude,valeur_fonciere,surface_reelle_bati,nombre_pieces_principales,prix_metre_carre,distance_moyenne,index_voisins
0,2015-1131554,2015-06-25,Vente,97136.0,97130,1,971,Maison,15.848188,-61.644636,1860.00,90.0,4.0,21,0.0,0.0
2,2015-1131605,2015-07-08,Vente,97136.0,97130,1,971,Maison,15.848572,-61.644027,1570.00,98.0,4.0,16,0.0,0.0
3,2015-1132114,2015-12-23,Vente,97136.0,97130,1,971,Maison,15.848833,-61.643867,1450.00,148.0,5.0,10,0.0,0.0
4,2015-1131405,2015-04-09,Vente,97137.0,97131,1,971,Maison,15.862501,-61.597274,720000.00,88.0,3.0,8182,0.0,0.0
5,2015-1132113,2015-12-30,Vente,97137.0,97131,1,971,Maison,15.862801,-61.586874,10671.43,38.0,3.0,281,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071014,2020-227595,2020-12-31,Vente,20600.0,2B120,94,2B,Maison,42.649121,9.443310,375000.00,138.0,5.0,2717,0.0,0.0
3071015,2020-227568,2020-11-16,Vente,20200.0,2B353,94,2B,Maison,42.714854,9.427883,180000.00,104.0,3.0,1731,0.0,0.0
3071016,2020-227575,2020-12-17,Vente,20200.0,2B305,94,2B,Maison,42.733560,9.460780,123500.00,33.0,1.0,3742,0.0,0.0
3071017,2020-227564,2020-10-14,Vente,20233.0,2B224,94,2B,Maison,42.840266,9.456825,265000.00,82.0,4.0,3232,0.0,0.0


In [65]:
data_predict = pd.DataFrame()
data_predict['latitude'] = [42.958421]
data_predict['longitude'] = [9.452980]
data_predict['surface_reelle_bati'] = [84.0]
data_predict['code_region'] = [94]
df_immo_maison[df_immo_maison.code_region == data_predict.code_region[0]]

,id_mutation,date_mutation,nature_mutation,code_postal,code_commune,code_region,code_departement,type_local,latitude,longitude,valeur_fonciere,surface_reelle_bati,nombre_pieces_principales,prix_metre_carre,distance_moyenne,index_voisins
439587,2015-285462,2015-12-31,Vente,20169.0,2A041,94,2A,Maison,41.368385,9.271691,3250000.0,160.0,5.0,20312,0.0,0.0
439588,2015-283415,2015-05-06,Vente,20169.0,2A041,94,2A,Maison,41.370691,9.267119,2400000.0,160.0,4.0,15000,0.0,0.0
439589,2015-285372,2015-12-30,Vente,20169.0,2A041,94,2A,Maison,41.376836,9.210876,4560000.0,214.0,3.0,21308,0.0,0.0
439591,2015-285192,2015-10-17,Vente,20169.0,2A041,94,2A,Maison,41.386177,9.200787,534880.0,70.0,3.0,7641,0.0,0.0
439594,2015-284831,2015-10-19,Vente,20169.0,2A041,94,2A,Maison,41.390734,9.170693,200000.0,143.0,6.0,1399,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071014,2020-227595,2020-12-31,Vente,20600.0,2B120,94,2B,Maison,42.649121,9.443310,375000.0,138.0,5.0,2717,0.0,0.0
3071015,2020-227568,2020-11-16,Vente,20200.0,2B353,94,2B,Maison,42.714854,9.427883,180000.0,104.0,3.0,1731,0.0,0.0
3071016,2020-227575,2020-12-17,Vente,20200.0,2B305,94,2B,Maison,42.733560,9.460780,123500.0,33.0,1.0,3742,0.0,0.0
3071017,2020-227564,2020-10-14,Vente,20233.0,2B224,94,2B,Maison,42.840266,9.456825,265000.0,82.0,4.0,3232,0.0,0.0


In [15]:
# Download model Balltree
from sklearn.externals import joblib

models = {}
regions = df_immo_maison.code_region.unique()
for k in range(len(regions)):
    start = time.time()
    name_model = 'model_balltree_region_' + str(regions[k]) + '.pkl'
    data = df_immo_maison[df_immo_maison.code_region==regions[k]]
    data = data.reset_index(drop=True)
    models = BallTree(data[['latitude', 'longitude']].values, leaf_size=2, metric='haversine')
    joblib_file = "/content/drive/MyDrive/data_pa/model_balltree/" + name_model
    joblib.dump(models, joblib_file)
    stop = time.time()
    print(stop-start)

0.1731560230255127
0.025856494903564453
0.01863241195678711
0.04418015480041504
0.7510645389556885
0.4581902027130127
0.4200716018676758
0.5809452533721924
1.1761455535888672
0.32863688468933105
0.7033684253692627
0.82108473777771
1.4275412559509277
1.013927698135376
0.8998818397521973
0.534332275390625
0.03934597969055176


In [152]:
data_predict = pd.DataFrame()
data_predict['latitude'] = [42.958421]
data_predict['longitude'] = [9.452980]
data_predict['surface_reelle_bati'] = [84.0]
data_predict['code_region'] = [94]

# load model balltree
from sklearn.externals import joblib
data = df_immo_maison[df_immo_maison.code_region == data_predict.code_region[0]]
data = data.reset_index(drop=True)
print(data[['latitude', 'longitude']])

joblib_file = "/content/drive/MyDrive/data_pa/model_balltree/model_balltree_region_"+str(data_predict.code_region[0])+".pkl"
model_load = joblib.load(joblib_file)

dist, indices = model_load.query(data_predict[['latitude','longitude']].values,k=10)
data_predict['distance_moyenne'] = np.mean(dist[:,1:]*6341,1)

print(dist)
print(indices)

a = pd.DataFrame()
a['prix_metre_carre'] = np.zeros(len(data_predict))
for i in range(1,10):
    a += pd.DataFrame(data.iloc[indices[:,i],:]['prix_metre_carre']).reset_index(drop=True)
a=a/10
data_predict['prix_moyen_cartier'] = a.values
data_predict

       latitude  longitude
0     41.368385   9.271691
1     41.370691   9.267119
2     41.376836   9.210876
3     41.386177   9.200787
4     41.390734   9.170693
...         ...        ...
5704  42.649121   9.443310
5705  42.714854   9.427883
5706  42.733560   9.460780
5707  42.840266   9.456825
5708  42.958421   9.452980

[5709 rows x 2 columns]
[[0.         0.00166155 0.001923   0.00192464 0.0024185  0.00276309
  0.00277995 0.00332932 0.00344079 0.00422668]]
[[5708 4290 4289 4291 3028  844 4292 1839 1838 3029]]


,latitude,longitude,surface_reelle_bati,code_region,distance_moyenne,prix_moyen_cartier
0,42.958421,9.45298,84.0,94,17.238727,2636.3


In [153]:
data = df_immo_maison[df_immo_maison.code_region == data_predict.code_region[0]]
data = data.reset_index(drop=True)
print(data[['latitude', 'longitude']])

models = BallTree(data[['latitude', 'longitude']].values, leaf_size=2, metric='haversine')
dist, indices = models.query(data_predict[['latitude','longitude']].values,k=10)

print(dist)
print(indices)

data_predict['distance_moyenne'] = np.mean(dist[:,1:]*6341,1)

a = pd.DataFrame()
a['prix_metre_carre'] = np.zeros(len(data_predict))
for i in range(1,10):
    a += pd.DataFrame(data.iloc[indices[:,i],:]['prix_metre_carre']).reset_index(drop=True)
a=a/10
data_predict['prix_moyen_cartier'] = a.values
data_predict

       latitude  longitude
0     41.368385   9.271691
1     41.370691   9.267119
2     41.376836   9.210876
3     41.386177   9.200787
4     41.390734   9.170693
...         ...        ...
5704  42.649121   9.443310
5705  42.714854   9.427883
5706  42.733560   9.460780
5707  42.840266   9.456825
5708  42.958421   9.452980

[5709 rows x 2 columns]
[[0.         0.00166155 0.001923   0.00192464 0.0024185  0.00276309
  0.00277995 0.00332932 0.00344079 0.00422668]]
[[5708 4290 4289 4291 3028  844 4292 1839 1838 3029]]


,latitude,longitude,surface_reelle_bati,code_region,distance_moyenne,prix_moyen_cartier
0,42.958421,9.45298,84.0,94,17.238727,2636.3


Prediction avec chargement des modéle

In [173]:
data_predict = pd.DataFrame()
data_predict['latitude'] = [42.958421]
data_predict['longitude'] = [9.452980]
data_predict['surface_reelle_bati'] = [84.0]
data_predict['code_region'] = [94]

# load model balltree
from sklearn.externals import joblib
data = df_immo_maison[df_immo_maison.code_region == data_predict.code_region[0]]
data = data.reset_index(drop=True)

joblib_file = "/content/drive/MyDrive/data_pa/model_balltree/model_balltree_region_"+str(data_predict.code_region[0])+".pkl"
model_load = joblib.load(joblib_file)

dist, indices = model_load.query(data_predict[['latitude','longitude']].values,k=10)
data_predict['distance_moyenne'] = np.mean(dist[:,1:]*6371)

print(dist)
print(indices)

a = pd.DataFrame()
a['prix_metre_carre'] = np.zeros(len(data_predict))
for i in range(1,10):
    a += pd.DataFrame(data.iloc[indices[:,i],:]['prix_metre_carre']).reset_index(drop=True)
a=a/10
data_predict['prix_moyen_cartier'] = a.values
data_predict

[[0.         0.00166155 0.001923   0.00192464 0.0024185  0.00276309
  0.00277995 0.00332932 0.00344079 0.00422668]]
[[5708 4290 4289 4291 3028  844 4292 1839 1838 3029]]


,latitude,longitude,surface_reelle_bati,code_region,distance_moyenne,prix_moyen_cartier
0,42.958421,9.45298,84.0,94,17.320285,2636.3


In [174]:
def getEncoderRegion(liste_region, code_region):
  for i in range(len(liste_region)):
    if liste_region[i] == code_region:
      return i
  return None

In [175]:
 df_immo_maison.code_region.unique()

array([ 1,  2,  3,  4, 11, 24, 27, 28, 32, 44, 52, 53, 75, 76, 84, 93, 94])

In [176]:
data_predict = data_predict[['latitude', 'longitude', 'surface_reelle_bati', 'code_region','prix_moyen_cartier']]
regions = df_immo_maison.code_region.unique()
encoder_region = getEncoderRegion(regions, data_predict.code_region[0])
print(encoder_region, data_predict.code_region[0])
data_predict['code_region'] = encoder_region
X = data_predict.values
X

    latitude  longitude  surface_reelle_bati  code_region  prix_moyen_cartier
0  42.958421    9.45298                 84.0           94              2636.3
16 94


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


array([[  42.958421,    9.45298 ,   84.      ,   16.      , 2636.3     ]])

In [172]:
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler

# Path Model
joblib_file = "/content/drive/MyDrive/data_pa/model_immo_maison_full_region_7err.pkl"
# Load Model
joblib_model = joblib.load(joblib_file)
#predict
joblib_model.predict(X)

array([16.82255952])

In [177]:
joblib_model.predict([ [ 15.848572, -61.644027,  98,         0,       18.8     ] ])
joblib_model.predict(X)

array([2808.60562771])